THIS NOTEBOOK IS FOR A DATA SCIENCE CAPSTONE PROJECT

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [4]:
print("week 2")

week 2


SEGMENTING AND CLUSTERING THE NEIGHBOURHOODS IN TORONTO

Import the necessary libraries

In [12]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from bs4 import BeautifulSoup
import csv

WEBSCRAPPING

In [13]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})

GET COLUMNS AND ROWS

In [14]:
headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

In [15]:
with open('Toronto.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

LOAD CSV BACK INTO A PANDAS DATAFRAME

In [16]:
df = pd.read_csv('Toronto.csv')
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [17]:

df.columns

Index(['Postcode', 'Borough', 'Neighbourhood\n'], dtype='object')

In [18]:
df.describe()

,Postcode,Borough,Neighbourhood
count,289,289,289
unique,180,12,210
top,M9V,Not assigned,Not assigned\n
freq,8,77,78


In [19]:
df.shape

(289, 3)

DATA CLEANING

rename columns 

In [20]:
df.rename(columns={'Neighbourhood\n':'Neighborhood','Postcode':'PostalCode'}, inplace=True)
df.head()
# Change Postcode to PostalCode

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [21]:
df['Neighborhood']=df['Neighborhood'].replace(to_replace='\n', value='', regex=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


remove data with 'NOT ASSIGNED' borough

In [22]:
df.drop(df[df['Borough'] == 'Not assigned'].index, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [23]:
df.shape

(212, 3)

Combining neighbourhoods into one postal code area

Test with PostalCode "M5A"

In [24]:
df.query('PostalCode == "M5A"')

,PostalCode,Borough,Neighborhood
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


Group the neighbourhoods

In [25]:

df_new=df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [26]:
df_new.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [27]:
df_new.query('Neighborhood == "Not assigned"')

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [28]:
#Replace "Not assigned" with the Borough name
df_new.loc[df_new.Neighborhood == 'Not assigned', "Neighborhood"] = df_new.Borough

In [29]:
df_new.query('PostalCode == "M7A"')

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [30]:
# Data was successfully replaced

Lastly, dispalce the number of rows

In [31]:
df_new.shape

(103, 3)

Write the new dataframe to a new CSV

In [32]:
df_new.to_csv('Toronto_New.csv')

FINAL CODE : WEEK 5

In [34]:
#import libraries

import requests
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  24.77 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  38.23 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  54.84 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.10 MB/s
vincent-0.4.4- 100% |###################

In [40]:
!pip install beautifulsoup4

Requirement not upgraded as not directly required: beautifulsoup4 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [41]:

!pip install lxml

Requirement not upgraded as not directly required: lxml in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [42]:

!pip install html5lib

Requirement not upgraded as not directly required: html5lib in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Requirement not upgraded as not directly required: webencodings in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)
Requirement not upgraded as not directly required: setuptools>=18.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from html5lib)


In [43]:
!pip install requests

Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests)
Requirement not upgraded as not directly required: certifi>=2017.4.17 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests)


In [35]:
from bs4 import BeautifulSoup

In [36]:
#Assign website link to a variable
wikipedia_link='https://www.zipcodestogo.com/New%20Jersey/'

In [37]:
#Use the function get from the requests library to download the Wikipedia page using the wikipedia_link as an argument. Assign the object to the variable postal_code_page.

#hint: requests.get()
source = requests.get(wikipedia_link).text
soup= BeautifulSoup(source,'lxml')
#print(soup.prettify())

In [38]:
table = soup.table
#print(table)
postheader = table.find_all('tr')
#print(postheader)

In [44]:
!pip install tabulate

Requirement not upgraded as not directly required: tabulate in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages


In [45]:
from tabulate import tabulate
table = soup.find_all('table')[1] 
df_list = pd.read_html(str(table))
#df_list[0]
type(df_list)
#print( tabulate(df_list[0], headers='keys', tablefmt='psql') )

data = pd.DataFrame(df_list[0])
type(data)
#data # 290*3

new_header = data.iloc[0] #grab the first row for the header
data = data[1:] #take the data less the header row
data.columns = new_header #set the header row as the df header

type(data) #289*3
len(data)
data.head()

,Zip Codes for the State of New Jersey,nan,nan,nan
1,Zip Code,City,County,Zip Code Map
2,07001,Avenel,Middlesex,View Map
3,07002,Bayonne,Hudson,View Map
4,07003,Bloomfield,Essex,View Map
5,07004,Fairfield,Essex,View Map


reassign the index

In [46]:
data = data.reset_index(drop=True)
len(data)
data.head()

,Zip Codes for the State of New Jersey,nan,nan,nan
0,Zip Code,City,County,Zip Code Map
1,07001,Avenel,Middlesex,View Map
2,07002,Bayonne,Hudson,View Map
3,07003,Bloomfield,Essex,View Map
4,07004,Fairfield,Essex,View Map


In [47]:

data.columns = data.iloc[0]
#data = data.reindex(data.index.drop(0)).reset_index(drop=True)
#data.columns.name = None
data.head()

,Zip Code,City,County,Zip Code Map
0,Zip Code,City,County,Zip Code Map
1,07001,Avenel,Middlesex,View Map
2,07002,Bayonne,Hudson,View Map
3,07003,Bloomfield,Essex,View Map
4,07004,Fairfield,Essex,View Map


Drop the column Zip Code as it is not needed

In [48]:

data_1 = data.drop('Zip Code', axis=1)

Drop the column Zip Code Map as it is not needed

In [50]:
data_2 = data_1.drop('Zip Code Map', axis=1)


Drop null values if any from the data frame

In [51]:

data_2.dropna()

,City,County
0,City,County
1,Avenel,Middlesex
2,Bayonne,Hudson
3,Bloomfield,Essex
4,Fairfield,Essex
5,Boonton,Morris
6,Caldwell,Essex
7,Caldwell,Essex
8,Carteret,Middlesex
9,Cedar Grove,Essex


In [52]:

data_2.head()
len(data_2)
data_2.head()

,City,County
0,City,County
1,Avenel,Middlesex
2,Bayonne,Hudson
3,Bloomfield,Essex
4,Fairfield,Essex



Make the first row as the column header

In [53]:
data_2 = data_2.iloc[1:]
data_2.head()

,City,County
1,Avenel,Middlesex
2,Bayonne,Hudson
3,Bloomfield,Essex
4,Fairfield,Essex
5,Boonton,Morris



Drop duplicates on the basis of City & County fields

In [54]:
data_2 = data_2.drop_duplicates(subset=['City','County'])
data_2.head()
len(data_2)

584


Drop row with City values 'Mc Afee' & 'Sussex' as it doesn't have geo coordinates

In [55]:
data_2 = data_2[data_2.City != 'Mc Afee']

In [56]:
len(data_2)

583

In [57]:
data_2 = data_2[data_2.County != 'Sussex' ]

Drop rows with City value of 'Short Hills' as it doesn't return nearby venues from FourSquare

In [58]:
data_2 = data_2[data_2.City != 'Short Hills']

In [59]:
data_2 = data_2.reset_index(drop=True)

In [60]:

print(len(data_2))
data_2.head()

557


,City,County
0,Avenel,Middlesex
1,Bayonne,Hudson
2,Bloomfield,Essex
3,Fairfield,Essex
4,Boonton,Morris


In [61]:
len(data_2)

557

In [62]:

data_2.head()

,City,County
0,Avenel,Middlesex
1,Bayonne,Hudson
2,Bloomfield,Essex
3,Fairfield,Essex
4,Boonton,Morris


In [63]:

column_names = ['Latitude', 'Longitude'] 
n_hood = pd.DataFrame(columns=column_names)
n_hood.shape

(0, 2)

In [64]:
for index, row in data_2.iterrows():
    try:
        address_1 = row['City'] 
        address = address_1+","+row['County']+","+"USA"
        #print(address)
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        #print(row['District'],address, latitude, longitude)
        n_hood = n_hood.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
        n_hood
        pass
    except ValueError as error_message:
        print("Error")
        pass
    except AttributeError:
        #print("Problem with data or cannot Geocode.")
        address = row['County']+","+"USA"
        print(address)
        geolocator = Nominatim()
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
        #print(address, latitude, longitude)
        n_hood = n_hood.append({'Latitude': latitude,'Longitude': longitude}, ignore_index=True)
        #print(row['District'],address, latitude, longitude)
        n_hood
        pass

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:6: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


Morris,USA


/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:21: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.




Concatenate the latitude and longitude values to the main dataframe to complete the results

In [69]:
len(n_hood)

557

In [70]:
df = pd.concat([data_2, n_hood[['Latitude', 'Longitude']]], axis=1)
df.shape

(557, 4)

In [71]:

df.head()

,City,County,Latitude,Longitude
0,Avenel,Middlesex,40.580382,-74.285147
1,Bayonne,Hudson,40.668714,-74.114309
2,Bloomfield,Essex,40.806767,-74.185423
3,Fairfield,Essex,40.883710,-74.305982
4,Boonton,Morris,40.902599,-74.407097


In [72]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['County'].unique()),
        df.shape[0]
    )
)

The dataframe has 20 boroughs and 557 neighborhoods.


In [73]:
#address = 'New York City, NY'
#address = 'Scarborough,Rouge,Malvern,Ontario,Canada'
address = 'New Jersey,USA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of New York City are 40.0757384, -74.4041622.


EXPLORE New Jersey

In [74]:
# create map of New York using latitude and longitude values
map_nj = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['County'], df['City']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nj) 
    
map_nj

In [75]:
essex_data = df[df['County'] == 'Essex'].reset_index(drop=True)
len(essex_data)
essex_data.head()


,City,County,Latitude,Longitude
0,Bloomfield,Essex,40.806767,-74.185423
1,Fairfield,Essex,40.883710,-74.305982
2,Caldwell,Essex,40.839822,-74.276537
3,Cedar Grove,Essex,40.851766,-74.229035
4,East Orange,Essex,40.767323,-74.204868


In [76]:
address = 'Essex, New Jersey'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Manhattan are 40.7913922, -74.2479444.


In [77]:
# create map of Manhattan using latitude and longitude values
map_essex = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(essex_data['Latitude'], essex_data['Longitude'], essex_data['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_essex)  
    
map_essex

In [78]:
CLIENT_ID = 'your-client-ID' # your Foursquare ID
CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret
VERSION = '20180924' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your-client-ID
CLIENT_SECRET:your-client-secret


In [79]:
#address = 'New York City, NY'
#address = 'Scarborough,Rouge,Malvern,Ontario,Canada'
address = 'New Jersey,USA'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))


name = 'Essex' # neighborhood name
print(name)

print('Latitude and longitude values of {} are {}, {}.'.format(name, 
                                                               latitude, 
                                                               longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of New York City are 40.0757384, -74.4041622.
Essex
Latitude and longitude values of Essex are 40.0757384, -74.4041622.


In [89]:

essex_data.loc[16, 'City']

'Newark'

In [90]:
neighborhood_latitude = essex_data.loc[16, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = essex_data.loc[16, 'Longitude'] # neighborhood longitude value

neighborhood_name = essex_data.loc[16, 'City'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Newark are 40.735657, -74.1723667.


In [91]:
# type your answer here
# type your answer here
CLIENT_ID = 'EV4PLNEYRE4D43TC2ZA5IXZVHOJFLMIUW1YGAGZ1OUGYLKTW'
CLIENT_SECRET = '3OYWIAFIWRXE4GWJFFRWINC5GXXFPOS3RWR3HNPHRAWONUP1'
VERSION = '20180922'
radius = 1000
LIMIT = 500

#create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=EV4PLNEYRE4D43TC2ZA5IXZVHOJFLMIUW1YGAGZ1OUGYLKTW&client_secret=3OYWIAFIWRXE4GWJFFRWINC5GXXFPOS3RWR3HNPHRAWONUP1&v=20180922&ll=40.735657,-74.1723667&radius=1000&limit=500'

In [92]:
results = requests.get(url).json()
#results

In [93]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [94]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#nearby_venues
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [95]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    #print(nearby_venues)
    return(nearby_venues)

In [96]:
essex_venues = getNearbyVenues(names=essex_data['City'],
                                   latitudes=essex_data['Latitude'],
                                   longitudes=essex_data['Longitude']
                                  )

Bloomfield
Fairfield
Caldwell
Cedar Grove
East Orange
Essex Fells
Glen Ridge
Livingston
Maplewood
Millburn
Montclair
Verona
Orange
West Orange
Roseland
South Orange
Newark
Belleville
Nutley
Irvington


In [97]:
essex_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Belleville,22,22,22,22,22,22
Bloomfield,2,2,2,2,2,2
Caldwell,33,33,33,33,33,33
Cedar Grove,28,28,28,28,28,28
East Orange,12,12,12,12,12,12
Essex Fells,1,1,1,1,1,1
Fairfield,2,2,2,2,2,2
Glen Ridge,14,14,14,14,14,14
Irvington,6,6,6,6,6,6


In [98]:
print('There are {} uniques categories.'.format(len(essex_venues['Venue Category'].unique())))

There are 132 uniques categories.


In [99]:
len(essex_data)

20

In [100]:

# one hot encoding
essex_onehot = pd.get_dummies(essex_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
essex_onehot['Neighborhood'] = essex_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [essex_onehot.columns[-1]] + list(essex_onehot.columns[:-1])
essex_onehot = essex_onehot[fixed_columns]

len(essex_onehot)

405

In [101]:
essex_onehot.shape

(405, 133)

In [102]:
essex_grouped = essex_onehot.groupby('Neighborhood').mean().reset_index()
len(essex_grouped)

20

In [103]:
essex_grouped

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Garden,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Classroom,Colombian Restaurant,Convenience Store,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Ethiopian Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Health & Beauty Service,Historic Site,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Latin American Restaurant,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Music School,Music Venue,Nail Salon,New American Restaurant,Non-Profit,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Women's Store,Yoga Studio
0,Belleville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.045455,0.000000,0.000000,0.0,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.0,0.045455,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.090909,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000
1,Bloomfield,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [104]:

num_top_venues = 5

for hood in essex_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = essex_grouped[essex_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Belleville----
                venue  freq
0  Italian Restaurant  0.09
1       Grocery Store  0.09
2      Sandwich Place  0.09
3   Convenience Store  0.09
4          Donut Shop  0.05


----Bloomfield----
                 venue  freq
0     Business Service   0.5
1       Cosmetics Shop   0.5
2  American Restaurant   0.0
3                 Pool   0.0
4             Pharmacy   0.0


----Caldwell----
                venue  freq
0         Pizza Place  0.12
1  Italian Restaurant  0.12
2              School  0.03
3  Seafood Restaurant  0.03
4         Coffee Shop  0.03


----Cedar Grove----
                venue  freq
0                Bank  0.07
1        Liquor Store  0.07
2          Bagel Shop  0.07
3  Italian Restaurant  0.07
4      Sandwich Place  0.04


----East Orange----
                 venue  freq
0  American Restaurant  0.17
1         Liquor Store  0.17
2   Chinese Restaurant  0.17
3                 Food  0.08
4       Sandwich Place  0.08


----Essex Fells----
                 venue 

In [105]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [106]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = essex_grouped['Neighborhood']

for ind in np.arange(essex_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(essex_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()
len(neighborhoods_venues_sorted)

20

In [107]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Belleville,Convenience Store,Italian Restaurant,Sandwich Place,Grocery Store,Cosmetics Shop,Shoe Store,Shopping Plaza,Salon / Barbershop,Martial Arts Dojo,Supplement Shop
1,Bloomfield,Business Service,Cosmetics Shop,Yoga Studio,French Restaurant,Gift Shop,Gastropub,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food & Drink Shop
2,Caldwell,Pizza Place,Italian Restaurant,Music School,Seafood Restaurant,School,Movie Theater,Chinese Restaurant,Clothing Store,Coffee Shop,Flower Shop
3,Cedar Grove,Italian Restaurant,Liquor Store,Bagel Shop,Bank,Yoga Studio,Dessert Shop,Coffee Shop,Sandwich Place,Salon / Barbershop,Ice Cream Shop
4,East Orange,American Restaurant,Chinese Restaurant,Liquor Store,Deli / Bodega,Fish & Chips Shop,Sandwich Place,Food,Seafood Restaurant,Recording Studio,Historic Site
5,Essex Fells,Lake,Gourmet Shop,Dog Run,Donut Shop,Ethiopian Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,French Restaurant
6,Fairfield,Historic Site,Sporting Goods Shop,Yoga Studio,Gift Shop,Gastropub,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food & Drink Shop
7,Glen Ridge,Gastropub,Bakery,Baseball Field,Convenience Store,Burger Joint,Mobile Phone Shop,Pharmacy,Sandwich Place,Health & Beauty Service,Shopping Mall
8,Irvington,Supermarket,Restaurant,Asian Restaurant,Moving Target,Chinese Restaurant,Yoga Studio,Fried Chicken Joint,Gastropub,Furniture / Home Store,Frozen Yogurt Shop
9,Livingston,Ice Cream Shop,Italian Restaurant,Pizza Place,Greek Restaurant,Deli / Bodega,Chinese Restaurant,Sandwich Place,Salon / Barbershop,Coffee Shop,Restaurant


In [108]:
essex_grouped

,Neighborhood,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beer Garden,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Classroom,Colombian Restaurant,Convenience Store,Cosmetics Shop,Cycle Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Dog Run,Donut Shop,Ethiopian Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Health & Beauty Service,Historic Site,Hobby Shop,Hockey Arena,Hotel,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Lake,Latin American Restaurant,Liquor Store,Locksmith,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Movie Theater,Moving Target,Music School,Music Venue,Nail Salon,New American Restaurant,Non-Profit,Park,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Women's Store,Yoga Studio
0,Belleville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.045455,0.000000,0.000000,0.0,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.0,0.045455,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.090909,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000
1,Bloomfield,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [109]:
# set number of clusters
kclusters = 5

essex_grouped_clustering = essex_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(essex_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 4, 0, 0, 0, 3, 2, 0, 0, 0], dtype=int32)

In [110]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Belleville,Convenience Store,Italian Restaurant,Sandwich Place,Grocery Store,Cosmetics Shop,Shoe Store,Shopping Plaza,Salon / Barbershop,Martial Arts Dojo,Supplement Shop
1,Bloomfield,Business Service,Cosmetics Shop,Yoga Studio,French Restaurant,Gift Shop,Gastropub,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food & Drink Shop
2,Caldwell,Pizza Place,Italian Restaurant,Music School,Seafood Restaurant,School,Movie Theater,Chinese Restaurant,Clothing Store,Coffee Shop,Flower Shop
3,Cedar Grove,Italian Restaurant,Liquor Store,Bagel Shop,Bank,Yoga Studio,Dessert Shop,Coffee Shop,Sandwich Place,Salon / Barbershop,Ice Cream Shop
4,East Orange,American Restaurant,Chinese Restaurant,Liquor Store,Deli / Bodega,Fish & Chips Shop,Sandwich Place,Food,Seafood Restaurant,Recording Studio,Historic Site
5,Essex Fells,Lake,Gourmet Shop,Dog Run,Donut Shop,Ethiopian Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,French Restaurant
6,Fairfield,Historic Site,Sporting Goods Shop,Yoga Studio,Gift Shop,Gastropub,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food & Drink Shop
7,Glen Ridge,Gastropub,Bakery,Baseball Field,Convenience Store,Burger Joint,Mobile Phone Shop,Pharmacy,Sandwich Place,Health & Beauty Service,Shopping Mall
8,Irvington,Supermarket,Restaurant,Asian Restaurant,Moving Target,Chinese Restaurant,Yoga Studio,Fried Chicken Joint,Gastropub,Furniture / Home Store,Frozen Yogurt Shop
9,Livingston,Ice Cream Shop,Italian Restaurant,Pizza Place,Greek Restaurant,Deli / Bodega,Chinese Restaurant,Sandwich Place,Salon / Barbershop,Coffee Shop,Restaurant


In [111]:
essex_merged = essex_data

# add clustering labels
essex_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
essex_merged = essex_data.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='City')

essex_merged.head() # check the last columns!

,City,County,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bloomfield,Essex,40.806767,-74.185423,0,Business Service,Cosmetics Shop,Yoga Studio,French Restaurant,Gift Shop,Gastropub,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food & Drink Shop
1,Fairfield,Essex,40.883710,-74.305982,4,Historic Site,Sporting Goods Shop,Yoga Studio,Gift Shop,Gastropub,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food & Drink Shop
2,Caldwell,Essex,40.839822,-74.276537,0,Pizza Place,Italian Restaurant,Music School,Seafood Restaurant,School,Movie Theater,Chinese Restaurant,Clothing Store,Coffee Shop,Flower Shop
3,Cedar Grove,Essex,40.851766,-74.229035,0,Italian Restaurant,Liquor Store,Bagel Shop,Bank,Yoga Studio,Dessert Shop,Coffee Shop,Sandwich Place,Salon / Barbershop,Ice Cream Shop
4,East Orange,Essex,40.767323,-74.204868,0,American Restaurant,Chinese Restaurant,Liquor Store,Deli / Bodega,Fish & Chips Shop,Sandwich Place,Food,Seafood Restaurant,Recording Studio,Historic Site


In [112]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(essex_merged['Latitude'], essex_merged['Longitude'], essex_merged['City'], essex_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [113]:
essex_merged.loc[essex_merged['Cluster Labels'] == 0, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bloomfield,Business Service,Cosmetics Shop,Yoga Studio,French Restaurant,Gift Shop,Gastropub,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,Food & Drink Shop
2,Caldwell,Pizza Place,Italian Restaurant,Music School,Seafood Restaurant,School,Movie Theater,Chinese Restaurant,Clothing Store,Coffee Shop,Flower Shop
3,Cedar Grove,Italian Restaurant,Liquor Store,Bagel Shop,Bank,Yoga Studio,Dessert Shop,Coffee Shop,Sandwich Place,Salon / Barbershop,Ice Cream Shop
4,East Orange,American Restaurant,Chinese Restaurant,Liquor Store,Deli / Bodega,Fish & Chips Shop,Sandwich Place,Food,Seafood Restaurant,Recording Studio,Historic Site
7,Livingston,Ice Cream Shop,Italian Restaurant,Pizza Place,Greek Restaurant,Deli / Bodega,Chinese Restaurant,Sandwich Place,Salon / Barbershop,Coffee Shop,Restaurant
8,Maplewood,Coffee Shop,Pizza Place,Park,Ice Cream Shop,Mexican Restaurant,American Restaurant,Salad Place,Restaurant,Colombian Restaurant,Diner
9,Millburn,Italian Restaurant,Ice Cream Shop,Pizza Place,Coffee Shop,New American Restaurant,Deli / Bodega,French Restaurant,Yoga Studio,Israeli Restaurant,Plaza
10,Montclair,Park,Bank,Train Station,Chinese Restaurant,Bookstore,Cosmetics Shop,Theater,Café,Athletics & Sports,Arts & Crafts Store
11,Verona,Deli / Bodega,Pharmacy,Juice Bar,Print Shop,Bus Stop,Café,Convenience Store,Cosmetics Shop,Gym / Fitness Center,Ice Cream Shop
12,Orange,Sandwich Place,Shoe Store,Bakery,Cosmetics Shop,Gym,Food,College Classroom,Clothing Store,Caribbean Restaurant,Shipping Store


Cluster 1 : Seems to be the most happening place with lots of restaurants, bus facilities, supermarkets etc.,
Cluster 2: Must be a place with a lake to relax and have some food

In [114]:
essex_merged.loc[essex_merged['Cluster Labels'] == 1, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Irvington,Supermarket,Restaurant,Asian Restaurant,Moving Target,Chinese Restaurant,Yoga Studio,Fried Chicken Joint,Gastropub,Furniture / Home Store,Frozen Yogurt Shop



Clutser 3: Good for a day's outing to see the old world charm

In [115]:
essex_merged.loc[essex_merged['Cluster Labels'] == 2, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Glen Ridge,Gastropub,Bakery,Baseball Field,Convenience Store,Burger Joint,Mobile Phone Shop,Pharmacy,Sandwich Place,Health & Beauty Service,Shopping Mall


Cluster 4: Must be a place for pub and baseball enthusiasts to hang around

In [117]:
essex_merged.loc[essex_merged['Cluster Labels'] == 3, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Essex Fells,Lake,Gourmet Shop,Dog Run,Donut Shop,Ethiopian Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,French Restaurant


Cluster 5: Place for shopping and eatout

In [118]:
essex_merged.loc[essex_merged['Cluster Labels'] == 4, essex_merged.columns[[0] + list(range(5, essex_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Fairfield,Historic Site,Sporting Goods Shop,Yoga Studio,Gift Shop,Gastropub,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food & Drink Shop
